In [1]:
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # ignore the GPU warning
tf.__version__

'2.6.0'

In [2]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [4]:
print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


In [5]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(32,32,3)),
    tf.keras.layers.Conv2D(32,(3,3),padding='same',activation='relu'), # valid is default
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(10),
])

model.summary()

model.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),
              optimizer = tf.keras.optimizers.Adam(lr=1e-4),
              metrics =["accuracy"]
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 5, 128)         73856     
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                2

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [6]:
model.fit(x_train,y_train,batch_size= 64,epochs = 3,verbose = 1)

Epoch 1/3
782/782 [==============================] - 92s 117ms/step - loss: 1.7043 - accuracy: 0.3868
Epoch 2/3
782/782 [==============================] - 91s 117ms/step - loss: 1.3708 - accuracy: 0.5142
Epoch 3/3
782/782 [==============================] - 93s 119ms/step - loss: 1.2544 - accuracy: 0.5600


In [7]:
model.evaluate(x_test,y_test,batch_size=64,verbose=1)

157/157 [==============================] - 5s 31ms/step - loss: 1.2076 - accuracy: 0.5747


[1.2075592279434204, 0.5746999979019165]

## Functional API

In [8]:
def my_model():
    inputs = tf.keras.Input(shape=(32,32,3))
    x = tf.keras.layers.Conv2D(32,3)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(64,5,padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Conv2D(128,3)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Flatten()(x)
    outputs = tf.keras.layers.Dense(10)(x)
    
    model =tf.keras.Model(inputs =inputs,outputs= outputs)
    
    return model

In [9]:
mymodel = my_model()

mymodel.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),
              optimizer = tf.keras.optimizers.Adam(lr=1e-4),
              metrics =["accuracy"]
)

In [10]:
mymodel.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 32)        128       
_________________________________________________________________
tf.nn.relu (TFOpLambda)      (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 64)        51264     
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 15, 64)        256   

In [11]:
mymodel.fit(x_train,y_train,batch_size= 64,epochs = 3,verbose = 1)

Epoch 1/3
782/782 [==============================] - 335s 428ms/step - loss: 1.3864 - accuracy: 0.5127
Epoch 2/3
782/782 [==============================] - 380s 486ms/step - loss: 0.9919 - accuracy: 0.6521
Epoch 3/3
782/782 [==============================] - 451s 577ms/step - loss: 0.8228 - accuracy: 0.7133


In [12]:
mymodel.evaluate(x_test,y_test,batch_size=64,verbose=1)

157/157 [==============================] - 21s 132ms/step - loss: 0.9635 - accuracy: 0.6614


[0.9635233283042908, 0.6614000201225281]

## Adding Regularization with L2 and Dropout

In [13]:
def my_model():
    inputs = tf.keras.Input(shape=(32,32,3))
    x = tf.keras.layers.Conv2D(32,3,padding= 'same',
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(64,5,padding='same',
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Conv2D(128,3,
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x) # regularization
    x = tf.keras.activations.relu(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64,
                               kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(10)(x)
    
    model =tf.keras.Model(inputs =inputs,outputs= outputs)
    
    return model

In [14]:
mymodel = my_model()

print(mymodel.summary())

mymodel.compile(loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),
              optimizer = tf.keras.optimizers.Adam(lr=1e-4),
              metrics =["accuracy"]
)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
tf.nn.relu_3 (TFOpLambda)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 16, 16, 64)        51264     
_________________________________________________________________
batch_normalization_4 (Batch (None, 16, 16, 64)        256 

In [15]:
mymodel.fit(x_train,y_train,batch_size= 64,epochs = 3,verbose = 1)

Epoch 1/3
782/782 [==============================] - 511s 650ms/step - loss: 4.3393 - accuracy: 0.4272
Epoch 2/3
782/782 [==============================] - 416s 532ms/step - loss: 3.4153 - accuracy: 0.5682
Epoch 3/3
782/782 [==============================] - 398s 509ms/step - loss: 2.9000 - accuracy: 0.6306


In [16]:
mymodel.evaluate(x_test,y_test,batch_size=64,verbose=1)

157/157 [==============================] - 22s 138ms/step - loss: 2.7840 - accuracy: 0.6166


[2.7840328216552734, 0.616599977016449]